In [6]:
import random
import networkx as nx

class SocNetMec:
    def __init__(self, G, T, k):
        self.G = G
        self.T = T
        self.k = k
        self.reports = {}
        self.get_reports()
        


    def __init(self, t):
        if t == 0: u = max(nx.degree_alg.degree_centrality(self.G), key = lambda x: nx.degree_alg.degree_centrality(self.G)[x] )
        else: u = random.choice(list(self.G.nodes()))
        auction = self.choose_auction_format(t)
        return u, auction

    def __invite(self, t, u, v, auction, prob, val):
        if prob(u, v):
            truthful_val = val(t, v)
            bv = truthful_val if auction == self.vcg_auction else random.uniform(2/3 * truthful_val, truthful_val)
            Sv = self.select_neighbors(v)
            return bv, Sv
        else:
            return None, None

    def get_reports(self):
        for edge in self.G.edges():
            node1, node2 = edge

        # Add node2 to the reports of node1
        if node1 not in self.reports:
            self.reports[node1] = set()
        self.reports[node1].add(node2)

        # Add node1 to the reports of node2
        if node2 not in self.reports:
            self.reports[node2] = set()
        self.reports[node2].add(node1)

    def choose_auction_format(self, t):
        if t % 2 == 0: return self.vcg_auction 
        else: return self.mudar_auction

    def run(self, t, prob, val):
        s, auction = self.__init(t)
        revenue = 0
        bids = {}
        for u in self.G.nodes:
            if u != s:
                # Call the valuation oracle (val) to get the bid for bidder u
                bid = val(u)
                bids[u] = bid  # Store the bid in the bids dictionary
                self.__invite(t, s, u, auction, prob, val)

        allocation, payments = auction(self.k, self.G.nodes, self.reports, bids)

        for bidder, payment in payments.items():
            revenue += payment

        return revenue

    def prob(self, u, v):
        diffusion_prob = min(0.25, 2 / max(self.G.degree[u], self.G.degree[v]))
        return random.uniform(0, diffusion_prob) >= 1 - diffusion_prob

    def select_neighbors(self, v):
        neighbors = list(self.G.neighbors(v))
        num_neighbors = len(neighbors)
        if num_neighbors <= self.k:
            return neighbors
        else:
            return random.sample(neighbors, self.k)
        
    def vcg_auction(self, k, seller_net, reports, bids):
        # Step 1: Calculate Payments
        payments = {}
        externalities = {}
        for bidder in seller_net:
            bids_except_bidder = {key: bids[key] for key in bids if key != bidder}
            sorted_bids_except_bidder = sorted(bids_except_bidder, key=bids_except_bidder.get, reverse=True)
            if bidder in sorted_bids_except_bidder[:k]:
                allocated_bidder = sorted_bids_except_bidder[:k].index(bidder)
                payment = sum(bids_except_bidder[key] for key in sorted_bids_except_bidder[:k]) - allocated_bidder
                payments[bidder] = payment
                externalities[bidder] = sum(bids_except_bidder[key] for key in sorted_bids_except_bidder[k:]) - k

        # Step 2: Calculate Allocation
        allocation = {bidder: bidder in sorted_bids_except_bidder[:k] for bidder in seller_net}

        # Step 3: Apply Neighbor Externalities
        for bidder, neighbor_bidders in reports.items():
            for neighbor_bidder in neighbor_bidders:
                if neighbor_bidder in externalities:
                    payments[bidder] += externalities[neighbor_bidder]

        return allocation, payments 
    
    def mudan_auction(self, k, seller_net, reports, bids):
        # Step 1: Calculate Payments
        payments = {}
        externalities = {}
        for bidder in seller_net:
            bids_except_bidder = {key: bids[key] for key in bids if key != bidder}
            sorted_bids_except_bidder = sorted(bids_except_bidder, key=bids_except_bidder.get, reverse=True)
            if bidder in sorted_bids_except_bidder[:k]:
                payment = sum(bids_except_bidder[key] for key in sorted_bids_except_bidder[:k])
                payments[bidder] = payment
                externalities[bidder] = sum(bids_except_bidder[key] for key in sorted_bids_except_bidder[k:]) - k

        # Step 2: Calculate Allocation
        allocation = {bidder: bidder in sorted_bids_except_bidder[:k] for bidder in seller_net}

        # Step 3: Apply Neighbor Externalities
        for bidder, neighbor_bidders in reports.items():
            for neighbor_bidder in neighbor_bidders:
                if neighbor_bidder in externalities:
                    payments[bidder] += externalities[neighbor_bidder]

        return allocation, payments
    
    def mudar_auction(self, k, seller_net, reports, bids):
        # Step 1: Calculate Random Allocation
        allocation = {bidder: False for bidder in seller_net}
        allocated_bidders = random.sample(seller_net, min(k, len(seller_net)))
        for bidder in allocated_bidders:
            allocation[bidder] = True

        # Step 2: Calculate Random Payments
        payments = {bidder: -bids[bidder] if allocation[bidder] else 0 for bidder in seller_net}

        return allocation, payments




In [7]:



def input_data():
    n = 100
    G = nx.Graph()

    with open('net_3', 'r') as f:
        for s in f.readlines():
            G.add_edge(s.split()[0], s.split()[1])
    k = 5
    T = 5000

    #for the oracle val
    val = dict()
    for t in range(T):
        val[t] = dict()
        for u in G.nodes():
            val[t][u] = random.randint(1, 100)
    
    #for the oracle prob
    p = dict()
    for u in G.nodes():
        p[u] = dict()
    for u in G.nodes():
        for v in G[u]:
            if v not in p[u]:
                t=min(0.25, 2/max(G.degree(u), G.degree(v)))
                p[u][v] = p[v][u] = random.uniform(0, t)
            
    return G, k, T, val, p

def prob(u, v):
    r = random.random()
    if r <= p[u][v]:
        return True
    return False

def valf(t, u):
    return val[t][u]

G, k, T, val, p = input_data()
snm=SocNetMec(G, k, T)
revenue = 0
for step in range(T):
    revenue += snm.run(step, prob, valf)
    print(revenue)
        
print(revenue)


TypeError: valf() missing 1 required positional argument: 'u'